# Visualizing Drawing - Artwork Pairs

- This notebook contains the code to visualize the manually annotated ddrawing artwork paris. The pairs can be either stored into a pdf or as seperate png files.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img
from tqdm import tqdm
import pickle

from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.gridspec as gridspec

In [2]:
# manual annotated drawing - artwork paris
annotations_by_hand = "./../../data/manual_annotation/annotated_drawing_artwork_pairs.csv"
# id to path mapping of drawings and artworks
uid2path = "./../../data/text_data/uid2path.pkl"
# the parent data folder
data_folder = "./../../data/"

In [3]:
# read the files
manually_annotated_df = pd.read_csv(annotations_by_hand)
with open(uid2path, "rb") as outfile:
    uid2path_df = pickle.load(outfile)

In [4]:
# select the pairs that are confirmed to have a connection as non ambiguious 
manually_annotated_df_nonambi = manually_annotated_df[manually_annotated_df["pair"]==True]
# select the original drawings by ignoring the style augmented drawings
manually_annotated_df_original = manually_annotated_df_nonambi[manually_annotated_df_nonambi["drawing_id"]==manually_annotated_df_nonambi["parent"]]
manually_annotated_df_ambi = manually_annotated_df[manually_annotated_df["pair"]==False]

In [5]:
all_names = uid2path_df["uid"].tolist()

In [8]:
nonambi_dict = {}

for rid, row in manually_annotated_df_original.iterrows():
    artname = row["artwork_id"]
    drawingname = row["drawing_id"]
    if artname not in nonambi_dict:
        nonambi_dict[artname] = []
    if drawingname != "2014_14-17_1764_THA_R_C":
        nonambi_dict[artname].append(drawingname)   
    
    
ambi_dict = {}

for rid, row in manually_annotated_df_ambi.iterrows():
    artname = row["artwork_id"]
    drawingname = row["drawing_id"]
    if artname not in ambi_dict:
        ambi_dict[artname] = []
    ambi_dict[artname].append(drawingname)   

In [ ]:
with PdfPages('./../../data/intermediate_storage/ambigious_drawing_artwork_pairs_v2.pdf') as pdf:
    itr = 0
    tot_ambi_pairs = len(ambi_dict)
    for auid, duids in tqdm(ambi_dict.items(), total=tot_ambi_pairs):
        num_drawings = len(duids)
        if auid in all_names:
            apath = uid2path_df.loc[uid2path_df['uid'] == auid, 'path'].iloc[0]
            dpaths = [uid2path_df.loc[uid2path_df['uid'] == duid, 'path'].iloc[0] for duid in duids] 
            
            fig = plt.figure(1, figsize=(11.69,8.27))
            outer = gridspec.GridSpec(1, 2, wspace=0.2, hspace=0.2)
            
            rows_per_subplot = 3
            # Compute Rows required
            cols_per_subplot = num_drawings // rows_per_subplot
            cols_per_subplot += num_drawings % rows_per_subplot

            inner_1 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=outer[0], wspace=0.1, hspace=0.1)
            
            artimge = img.imread(data_folder + apath)
            ax = plt.Subplot(fig, inner_1[0])
            ax.imshow(artimge)
            ax.set_title(auid)
            ax.axis(False)
            fig.add_subplot(ax) 
            
            inner = gridspec.GridSpecFromSubplotSpec(rows_per_subplot, cols_per_subplot,
                                                         subplot_spec=outer[1], wspace=0.1,
                                                         hspace=0.1)
            
            for j in range(num_drawings):
                duid = duids[j]
                dimage = img.imread(data_folder + dpaths[j])
                ax = plt.Subplot(fig, inner[j])
                ax.imshow(dimage)
                ax.set_title(duid, size=5)
                ax.axis(False)
                fig.add_subplot(ax)
            
            itr += 1
            fig.suptitle("Pair: " + str(itr), fontsize=30)
            pdf.savefig()
            plt.close()
                          
    d = pdf.infodict()
    d['Title'] = 'Ambigious Drawing Artwork Pairs'
    d['Author'] = u'Ravinithesh'

In [ ]:
with PdfPages('./../../data/intermediate_storage/good_drawing_artwork_pairs_v2.pdf') as pdf:
    itr = 0
    tot_nonambi_pairs = len(nonambi_dict)
    for auid, duids in tqdm(nonambi_dict.items(), total=tot_nonambi_pairs):
        num_drawings = len(duids)
        apath = uid2path_df.loc[uid2path_df['uid'] == auid, 'path'].iloc[0]
        dpaths = [uid2path_df.loc[uid2path_df['uid'] == duid, 'path'].iloc[0] for duid in duids]

        fig = plt.figure(1, figsize=(11.69,8.27))
        outer = gridspec.GridSpec(1, 2, wspace=0.2, hspace=0.2)

        rows_per_subplot = 3
        if len(duids) > 6:
            rows_per_subplot = 6
        # Compute Rows required
        cols_per_subplot = num_drawings // rows_per_subplot
        cols_per_subplot += num_drawings % rows_per_subplot

        inner_1 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=outer[0], wspace=0.1, hspace=0.1)

        artimge = img.imread(data_folder + apath)
        ax = plt.Subplot(fig, inner_1[0])
        ax.imshow(artimge)
        ax.set_title(auid)
        ax.axis(False)
        fig.add_subplot(ax) 

        inner = gridspec.GridSpecFromSubplotSpec(rows_per_subplot, cols_per_subplot,
                                                     subplot_spec=outer[1], wspace=0.1,
                                                     hspace=0.1)

        for j in range(num_drawings):
            duid = duids[j]
            dimage = img.imread(data_folder + dpaths[j])
            ax = plt.Subplot(fig, inner[j])
            if len(dimage.shape) < 3:
                ax.imshow(dimage, cmap="gray")
            else:
                ax.imshow(dimage)
            ax.set_title(duid, size=5)
            ax.axis(False)
            fig.add_subplot(ax)

        itr += 1
        fig.suptitle("Pair: " + str(itr), fontsize=30)
        pdf.savefig()
        plt.close()

In [9]:
tot_nonambi_pairs = len(nonambi_dict)
for auid, duids in tqdm(nonambi_dict.items(), total=tot_nonambi_pairs):
    num_drawings = len(duids)
    apath = uid2path_df.loc[uid2path_df['uid'] == auid, 'path'].iloc[0]
    dpaths = [uid2path_df.loc[uid2path_df['uid'] == duid, 'path'].iloc[0] for duid in duids]

    fig = plt.figure(1, figsize=(11.69,8.27), dpi=200)
    outer = gridspec.GridSpec(1, 2, wspace=0.2, hspace=0.2)

    
    if len(duids) == 1:
        rows_per_subplot = 1
    elif len(duids) == 2:
        rows_per_subplot = 2
    elif len(duids) > 6:
        rows_per_subplot = 6
    else:
        rows_per_subplot = 3
    
    # Compute Rows required
    if len(duids) == 1:
        cols_per_subplot = 1
    else:    
        cols_per_subplot = num_drawings // rows_per_subplot
        cols_per_subplot += num_drawings % rows_per_subplot

    inner_1 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=outer[0], wspace=0.1, hspace=0.1)

    artimge = img.imread(data_folder + apath)
    ax = plt.Subplot(fig, inner_1[0])
    ax.imshow(artimge)
    ax.set_title(auid)
    ax.axis(False)
    fig.add_subplot(ax) 
    
    inner = gridspec.GridSpecFromSubplotSpec(rows_per_subplot, cols_per_subplot,
                                                 subplot_spec=outer[1], wspace=0.1,
                                                 hspace=0.1)

    for j in range(num_drawings):
        duid = duids[j]
        dimage = img.imread(data_folder + dpaths[j])
        ax = plt.Subplot(fig, inner[j])
        if len(dimage.shape) < 3:
            ax.imshow(dimage, cmap="gray")
        else:
            ax.imshow(dimage)
        ax.set_title(duid, size=5)
        ax.axis(False)
        fig.add_subplot(ax)

    # fig.suptitle("Pair: " + str(itr), fontsize=30)
    fig.savefig("./../../data/intermediate_storage/annotated_pairs_images/" + auid + ".png", bbox_inches="tight")
    plt.close()

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:42<00:00,  1.03s/it]
